In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_3262921/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp = df_tmp[4000:5000]
df_tmp 

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
19708,SAN MIGUEL SAN MIGUEL DE AZAPA LICEO AGRICOLA,ARICA,None,,
19709,CAM. VECINAL A FUNDO SAN FELIPE PARCELA 4,LA CRUZ,None,,
19715,C. VICTORIA 1560,PUCON,None,,
19720,MAHUIDANCHE PALO DE ROSA,PUENTE ALTO,None,,
19726,STA. ISABEL SANTA ISABEL CON VASCO SOLAR SIN N...,CHIMBARONGO,None,,
...,...,...,...,...,...
24183,Manuel H Alegría 1108,CORONEL,None,,
24188,TIERRA 1654 CASA,SAN PEDRO DE LA PAZ,None,,
24199,PABLO NERUDA CASA 590,LLANQUIHUE,None,,
24205,PEDRO AGUIRRE CERDA AL FINAL DEL PASAJE,FRESIA,None,,


In [6]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

ddata = dd.from_pandas(df_tmp, npartitions=30)

# Sample meta dataframe to indicate the structure of the output
meta = pd.DataFrame({
    "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
    "COMUNA DOMICILIO": pd.Series(dtype='str'),
    "NUEVA DIRECCION": pd.Series(dtype='str'),
    "X": pd.Series(dtype='float'),
    "Y": pd.Series(dtype='float')
})

def apply_validate_address(df_chunk):
    # Handle missing values before passing to validate_address
    df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
    
    # Apply the validation function to each row and unpack the results
    df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
        lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
        axis=1
    )
    return df_chunk

# Map the function to each partition, with the meta keyword to define the structure
res = ddata.map_partitions(apply_validate_address, meta=meta).compute()

['LAUTARO', '10208', 'CASA', 'ESQUINA', 'SAN RAMON']  ------  LAUTARO 10208 CASA ESQUINA SAN RAMON  -------  Lautaro, Padre C. Díaz, Maipo, Buin, Provincia de Maipo, Región Metropolitana de Santiago, Chile
['9', 'JULIO', '34', 'POBLACION', 'DIEGO', 'PORTALES', 'LLAILLAY']  ------  9 DE JULIO 34 POBLACION DIEGO PORTALES LLAILLAY  -------  Julio Zegers, Población El Aserrín, Villarrica, Provincia de Cautín, Región de la Araucanía, 4930611, Chile
['PASO', 'NEVADO', 'PARCELA', '#', '18', 'SAN CLEMENTE']  ------  PASO NEVADO SN  PARCELA # 18 SAN CLEMENTE CERCA DE LA CANCHA PASO NEVADO COMO 2 CUADRA DE LA CANCHA TALCA  -------  Chile, Barrio PAI CUE, Municipio de Paso de la Patria, Departamento San Cosme, Corrientes, 3904, Argentina
['LOS', 'CASTAÑOS', 'CALLE', 'EL CARMEN', 'PARADERO', '10', 'CALERA', 'TANGO']  ------  LOS CASTAÑOS CALLE EL CARMEN PARADERO 10 CALERA DE TANGO  -------  Los Castaños, Los Bajos de San Agustín, Calera de Tango, Provincia de Maipo, Región Metropolitana de Santiag

In [7]:
res

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
19708,SAN MIGUEL SAN MIGUEL DE AZAPA LICEO AGRICOLA,ARICA,None,NaN,NaN
19709,CAM. VECINAL A FUNDO SAN FELIPE PARCELA 4,LA CRUZ,"Camino Vecinal Parcela San Felipe, Campanario,...",-32.853348,-71.214405
19715,C. VICTORIA 1560,PUCON,None,NaN,NaN
19720,MAHUIDANCHE PALO DE ROSA,PUENTE ALTO,None,NaN,NaN
19726,STA. ISABEL SANTA ISABEL CON VASCO SOLAR SIN N...,CHIMBARONGO,"PMGD PFV Santa Elizabeth, Rengo, Provincia de ...",-34.313358,-70.907370
...,...,...,...,...,...
24183,Manuel H Alegría 1108,CORONEL,"Banco Estado, Germán Riesco, Población Bombero...",-34.440630,-71.074315
24188,TIERRA 1654 CASA,SAN PEDRO DE LA PAZ,"Tierra, Caleta Boca Sur, San Pedro de la Paz, ...",-36.825616,-73.146320
24199,PABLO NERUDA CASA 590,LLANQUIHUE,"Pablo Neruda, Villa Federico Werner, Llanquihu...",-41.255520,-73.009479
24205,PEDRO AGUIRRE CERDA AL FINAL DEL PASAJE,FRESIA,"Pedro Aguirre Cerda, Provincia de Santiago, Re...",-33.493398,-70.676891


In [8]:
print("A")

A


In [9]:
res.to_csv('./data/out/ad5.csv')